## Save model

In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, optimizers, Sequential, metrics
from tensorflow import keras

In [2]:
def preprocess(x, y):
    """x is a simple image, not a batch"""
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x, y

batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
# print('datasets:', x.shape, y.shape, x.min(), y.min())

db = tf.data.Dataset.from_tensor_slices((x, y))
db = db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz)

class MyDense(layers.Layer):
    def __init__(self, inp_dim, outp_dim):
        super(MyDense, self).__init__()
        self.kernel = self.add_variable('w',[inp_dim, outp_dim])
        self.bias = self.add_variable('b', [outp_dim])
    def call(self, inputs, training=None):
        out = inputs @ self.kernel + self.bias
        return out
    
class MyModel(keras.Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc1 = MyDense(28*28, 256)
        self.fc2 = MyDense(256, 128)
        self.fc3 = MyDense(128, 64)
        self.fc4 = MyDense(64, 32)
        self.fc5 = MyDense(32, 10)
    def call(self, inputs, training=None):
        x = self.fc1(inputs)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = tf.nn.relu(x)
        x = self.fc3(x)
        x = tf.nn.relu(x)
        x = self.fc4(x)
        x = tf.nn.relu(x)
        x = self.fc5(x)
        
        return x
    
# 创建神经网络 network
network = MyModel()
network.compile(optimizer=optimizers.Adam(lr=0.01),
               loss=tf.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
network.fit(db, epochs=3, validation_data=ds_val, validation_freq=2)
network.evaluate(ds_val)


Epoch 1/3
469/469 [==============================] - 3s 6ms/step - loss: 0.2827 - accuracy: 0.8368
Epoch 2/3
469/469 [==============================] - 2s 5ms/step - loss: 0.1388 - accuracy: 0.9576 - val_loss: 0.1470 - val_accuracy: 0.9622
Epoch 3/3
79/79 [==============================] - 0s 3ms/step - loss: 0.1222 - accuracy: 0.9697


[0.12223819872663545, 0.9697]

#### 保存 network 的参数到文件weights1.ckpt

In [5]:

# 保存 network 的******参数******到文件
network.save_weights('/home/kukafee/workspace/save_model/weights1.ckpt')
# 打印信息
print('Saved weights !')
# 删除网络
del network
# 重新创建神经网络
network = MyModel()
network.compile(optimizer=optimizers.Adam(lr=0.01),
               loss=tf.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
# 加载文件中的数据（保存有链接权重的数据）
network.load_weights('/home/kukafee/workspace/save_model/weights1.ckpt')
# 打印信息
print('Loaded weights !')
network.evaluate(ds_val)


Saved weights !
Loaded weights !
79/79 [==============================] - 0s 4ms/step - loss: 0.1298 - accuracy: 0.9582


[0.12983709347251615, 0.9636]

---

---

#### 保存 network1 的所有信息到文件model.h5

In [11]:
def preprocess(x, y):
    """x is a simple image, not a batch"""
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = tf.reshape(x, [28*28])
    y = tf.cast(y, dtype=tf.int32)
    y = tf.one_hot(y, depth=10)
    return x, y

batchsz = 128
(x, y), (x_val, y_val) = datasets.mnist.load_data()
# print('datasets:', x.shape, y.shape, x.min(), y.min())

db = tf.data.Dataset.from_tensor_slices((x, y))
db = db.map(preprocess).shuffle(60000).batch(batchsz)
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_val = ds_val.map(preprocess).batch(batchsz)

network1 = Sequential([   
    layers.Dense(256, activation=tf.nn.relu),    # [b, 784] => [b, 256]
    layers.Dense(128, activation=tf.nn.relu),    # [b, 256] => [b, 128]
    layers.Dense(64, activation=tf.nn.relu),     # [b, 128] => [b, 64]
    layers.Dense(32, activation=tf.nn.relu),     # [b, 64] => [b, 32]
    layers.Dense(10)                             # [b, 32] => [b, 10]
])
network1.build(input_shape=(None, 28*28))
network1.summary()
# 创建神经网络 network
network1.compile(optimizer=optimizers.Adam(lr=0.01),
               loss=tf.losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])
network1.fit(db, epochs=3, validation_data=ds_val, validation_freq=2)
network1.evaluate(ds_val)


# 保存 network 的******所有信息******到文件
network1.save('/home/kukafee/workspace/save_model/model.h5')
print('Saved total model !')
del network1
print('Load model from file:')
network1 = tf.keras.models.load_model('/home/kukafee/workspace/save_model/model.h5')
print('Loaded the model from file !')
# network1.summary()
# ????????????????????????????????????
x_val = tf.cast(x_val, dtype=tf.float32) / 255.
x_val = tf.reshape(x_val, [-1, 28*28])
y_val = tf.cast(y_val, dtype=tf.int32)
y_val = tf.one_hot(y_val, depth=10)
network1.evaluate(x_val, y_val)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_45 (Dense)             multiple                  200960    
_________________________________________________________________
dense_46 (Dense)             multiple                  32896     
_________________________________________________________________
dense_47 (Dense)             multiple                  8256      
_________________________________________________________________
dense_48 (Dense)             multiple                  2080      
_________________________________________________________________
dense_49 (Dense)             multiple                  330       
Total params: 244,522
Trainable params: 244,522
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
469/469 [==============================] - 2s 5ms/step - loss: 0.2762 - accuracy: 0.8376
Epoch 2/3
469/4

W0617 16:01:14.101134 140530799548224 hdf5_format.py:266] Sequential models without an `input_shape` passed to the first layer cannot reload their optimizer state. As a result, your model isstarting with a freshly initialized optimizer.


Saved total model !
Load model from file:
Loaded the model from file !


ValueError: Unknown entry in loss dictionary: class_name. Only expected following keys: ['output_1']

---

#### 